In [40]:
import requests
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.checkpoint.memory import InMemorySaver

In [41]:
load_dotenv()

True

In [42]:
model = ChatGroq(model="llama-3.1-8b-instant")
API_KEY = os.getenv("TMDB_API_KEY")

In [43]:
class MovieState(TypedDict):
    movie_name : str
    original_title : str
    description : str
    release_date : str
    language : str
    summary : str

In [44]:
def fetch_movie(state : MovieState):
    url = "https://api.themoviedb.org/3/search/movie"
    movie_name = state["movie_name"]

    params = {
        "api_key": API_KEY,
        "query": movie_name,
        "language": "en-US"
    }
    response = requests.get(url=url, params=params).json()

    description = ""
    release_date = ""
    for result in response["results"]:
        description = result.get("overview","")
        release_date = result.get("release_date","")
        original_language = result.get("original_language","")
        original_title = result.get("original_title","")
        break

    return {"description" : description, "original_title" : original_title, "release_date" : release_date, "language" : original_language}

In [45]:
def generate_explaination(state : MovieState):
    context = f"""
        MovieName : {state["movie_name"]}
        original title : {state["original_title"]}
        MovieReleaseDate : {state["release_date"]}
        language : {state["language"]}
        description of movie : {state["description"]}"""
    
    prompt = ChatPromptTemplate([
        ("system" , "You are a llm with a great knowledge on movies"),
        ("human", "Generate a breif summary on the context of the movie given \n\n{context}")
    ])
    parser = StrOutputParser()

    output = prompt | model | parser
    return {"summary" : output.invoke({"context":context})}

In [46]:
graph = StateGraph(MovieState)

graph.add_node("fetch_movie", fetch_movie)
graph.add_node("generate_explaination", generate_explaination)

graph.add_edge(START, "fetch_movie")
graph.add_edge("fetch_movie", "generate_explaination")
graph.add_edge("generate_explaination", END)

checkpointer = InMemorySaver()
workflow = graph.compile(checkpointer=checkpointer)

In [47]:
config1 = {"configurable":{"thread_id":"1"}}
initial_state = {"movie_name" : "KGF chapter 2"}

workflow.invoke(initial_state,config=config1)

{'movie_name': 'KGF chapter 2',
 'original_title': 'ಕೆ.ಜಿ.ಎಫ್: Chapter 2',
 'description': 'The blood-soaked land of Kolar Gold Fields (KGF) has a new overlord now - Rocky, whose name strikes fear in the heart of his foes. His allies look up to Rocky as their Savior, the government sees him as a threat to law and order; enemies are clamoring for revenge and conspiring for his downfall. Bloodier battles and darker days await as Rocky continues on his quest for unchallenged supremacy.',
 'release_date': '2022-04-14',
 'language': 'kn',
 'summary': "Here's a brief summary of the context of the movie KGF Chapter 2:\n\n**Context:** The movie is set in the 1980s in the Kolar Gold Fields (KGF) region of India. The film is a sequel to the first installment of KGF, where Rocky (the protagonist) has emerged as the new ruler of KGF. The story revolves around Rocky's quest for unchallenged supremacy, while facing opposition from the government, enemies, and internal conflicts within his own group 

In [48]:
workflow.get_state(config=config1)

StateSnapshot(values={'movie_name': 'KGF chapter 2', 'original_title': 'ಕೆ.ಜಿ.ಎಫ್: Chapter 2', 'description': 'The blood-soaked land of Kolar Gold Fields (KGF) has a new overlord now - Rocky, whose name strikes fear in the heart of his foes. His allies look up to Rocky as their Savior, the government sees him as a threat to law and order; enemies are clamoring for revenge and conspiring for his downfall. Bloodier battles and darker days await as Rocky continues on his quest for unchallenged supremacy.', 'release_date': '2022-04-14', 'language': 'kn', 'summary': "Here's a brief summary of the context of the movie KGF Chapter 2:\n\n**Context:** The movie is set in the 1980s in the Kolar Gold Fields (KGF) region of India. The film is a sequel to the first installment of KGF, where Rocky (the protagonist) has emerged as the new ruler of KGF. The story revolves around Rocky's quest for unchallenged supremacy, while facing opposition from the government, enemies, and internal conflicts withi

In [49]:
list(workflow.get_state_history(config=config1))

[StateSnapshot(values={'movie_name': 'KGF chapter 2', 'original_title': 'ಕೆ.ಜಿ.ಎಫ್: Chapter 2', 'description': 'The blood-soaked land of Kolar Gold Fields (KGF) has a new overlord now - Rocky, whose name strikes fear in the heart of his foes. His allies look up to Rocky as their Savior, the government sees him as a threat to law and order; enemies are clamoring for revenge and conspiring for his downfall. Bloodier battles and darker days await as Rocky continues on his quest for unchallenged supremacy.', 'release_date': '2022-04-14', 'language': 'kn', 'summary': "Here's a brief summary of the context of the movie KGF Chapter 2:\n\n**Context:** The movie is set in the 1980s in the Kolar Gold Fields (KGF) region of India. The film is a sequel to the first installment of KGF, where Rocky (the protagonist) has emerged as the new ruler of KGF. The story revolves around Rocky's quest for unchallenged supremacy, while facing opposition from the government, enemies, and internal conflicts with

In [50]:
config1 = {"configurable":{"thread_id":"1"}}
initial_state = {"movie_name" : "Lucky the racer"}


workflow.invoke(initial_state, config=config1)

{'movie_name': 'Lucky the racer',
 'original_title': 'రేసుగుర్రం',
 'description': 'Ram and Lakshman, aka Lucky, are brothers who are polar opposites in nature. Their mother wishes to see them united.',
 'release_date': '2014-04-11',
 'language': 'te',
 'summary': 'Here\'s a brief summary of the context of the movie "Lucky the Racer" (రేసుగుర్రం):\n\n"Lucky the Racer" is a 2014 Telugu-language drama film that revolves around the lives of two brothers, Ram and Lakshman, also known as Lucky. They are vastly different in nature, personality, and possibly even interests. Their mother, who is the central figure in their lives, desires to see her sons come together and unite as brothers. The movie likely explores their contrasting personalities, their relationships, and the efforts of their mother to bring them closer.'}

In [51]:
workflow.get_state(config=config1)

StateSnapshot(values={'movie_name': 'Lucky the racer', 'original_title': 'రేసుగుర్రం', 'description': 'Ram and Lakshman, aka Lucky, are brothers who are polar opposites in nature. Their mother wishes to see them united.', 'release_date': '2014-04-11', 'language': 'te', 'summary': 'Here\'s a brief summary of the context of the movie "Lucky the Racer" (రేసుగుర్రం):\n\n"Lucky the Racer" is a 2014 Telugu-language drama film that revolves around the lives of two brothers, Ram and Lakshman, also known as Lucky. They are vastly different in nature, personality, and possibly even interests. Their mother, who is the central figure in their lives, desires to see her sons come together and unite as brothers. The movie likely explores their contrasting personalities, their relationships, and the efforts of their mother to bring them closer.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e87ef-5ff9-6e33-8006-0cf3cc7337d2'}}, metadata={'source': 'loo

In [52]:
list(workflow.get_state_history(config=config1))

[StateSnapshot(values={'movie_name': 'Lucky the racer', 'original_title': 'రేసుగుర్రం', 'description': 'Ram and Lakshman, aka Lucky, are brothers who are polar opposites in nature. Their mother wishes to see them united.', 'release_date': '2014-04-11', 'language': 'te', 'summary': 'Here\'s a brief summary of the context of the movie "Lucky the Racer" (రేసుగుర్రం):\n\n"Lucky the Racer" is a 2014 Telugu-language drama film that revolves around the lives of two brothers, Ram and Lakshman, also known as Lucky. They are vastly different in nature, personality, and possibly even interests. Their mother, who is the central figure in their lives, desires to see her sons come together and unite as brothers. The movie likely explores their contrasting personalities, their relationships, and the efforts of their mother to bring them closer.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e87ef-5ff9-6e33-8006-0cf3cc7337d2'}}, metadata={'source': 'lo